<a href="https://colab.research.google.com/github/Bonnnana/Introduction-to-Data-Science/blob/main/Lab/Lab4_(20_12_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [44]:
# Add as many imports as you need.
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments


# Laboratory Exercise - Run Mode (8 points)

## Introduction
This laboratory assignment's primary objective is to fine-tune a pre-trained language model for binary classification on a dataset consisting of wine reviews. The dataset contains two attributes: **description** and **points**. The description is a brief text describing the wine and the points represent a quality metric ranging from 1 to 100. If some wine has at least 90 points it is considered **exceptional**. Your task involves predicting if some wine is **exceptional** based on its review.

## The Wine Reviews Dataset

Load the dataset using the `datasets` library.

In [6]:
# Write your code here. Add as many boxes as you need.
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [22]:
from datasets import load_dataset

In [29]:
dataset = load_dataset("csv", data_files="wine-reviews.csv")
dataset

DatasetDict({
    train: Dataset({
        features: ['description', 'points'],
        num_rows: 10000
    })
})

## Target Extraction
Extract the target **exceptional** for each wine review. If some wine has at least 90 points it is considered **exceptional**.

In [30]:
# Write your code here. Add as many boxes as you need.
def label_exceptional(sample):
    sample['label'] = int(sample['points'] >= 90)
    return sample
#vrakja 1 ili 0

In [31]:
dataset['train'] = dataset['train'].map(label_exceptional)  #se prakja po eden red

In [32]:
print(dataset['train'][:5])

{'description': ["Translucent in color, silky in the mouth, this Mendocino-grown Pinot shows ripe, forward flavors and crisp acidity. You'll find pie-filling cherries and rhubarb, with a cola and root beer soda finish.", 'On the palate, this wine is rich and complex, with flavors of dried cherry, cola, licorice, red currant and sandalwood that finish bone dry. It has lots of acidity and some sandpaper-like tannins. This should begin to develop bottle-age notes after 2016.', "The producer blends 57% Chardonnay from the Maldonado Vineyard with 39% Sauvignon Blanc and 4% Viognier for this fresh, floral white. On the palate, apple and lemon flavors are buttressed by a core of laser-like acidity. A fresh pear note lasts long into the finish, proving the wine's ability to be both creamy and crisp.", "Pure Baga in all its glory, packed with dry and dark tannins and a brooding character that is mitigated by the rich dark plum and blackberry jelly flavors. It's a wine that is probably at its pe

In [33]:
dataset['train'] = dataset['train'].remove_columns("points")
dataset = dataset.rename_column("description", "text")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})

## Dataset Splitting
Partition the dataset into training and testing sets with an 80:20 ratio.


In [34]:
# Write your code here. Add as many boxes as you need.
dataset = dataset["train"].train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [35]:
dataset["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None)}

## Tokenization
Tokenize the texts using the `AutoTokenizer` class.

In [36]:
# Write your code here. Add as many boxes as you need.
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_texts = tokenizer(dataset["train"]["text"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [37]:
input = tokenizer("Hi, how are you?")
input

{'input_ids': [101, 7632, 1010, 2129, 2024, 2017, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [38]:
tokenizer.decode([102])

'[SEP]'

In [39]:
tokenizer.convert_ids_to_tokens(input["input_ids"])

['[CLS]', 'hi', ',', 'how', 'are', 'you', '?', '[SEP]']

In [40]:
def tokenize(sample):
    return tokenizer(sample["text"], truncation=True)

In [41]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [42]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [43]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [103]:
tokenized_dataset['train'].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

## Fine-tuning a Pre-trained Language Model for Classification
Fine-tune a pre-trained language model for classification on the given dataset.

Define the model using the `AutoModelForSequenceClassification` class.

In [46]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define the traning parameters using the `TrainingArguments` class.

In [45]:
# Write your code here. Add as many boxes as you need.
training_args = TrainingArguments(
    output_dir="trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=128,  # batch size for training
    per_device_eval_batch_size=128,  # batch size for evaluation
    metric_for_best_model="f1",
)

Define the training using the `Trainer` class.

In [48]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [105]:
# Write your code here. Add as many boxes as you need.
from evaluate import load
import numpy as np

metric = load("f1")

In [106]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels, average="binary")

In [107]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [108]:
import torch

In [109]:
torch.cuda.is_available()


True

In [110]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [111]:
device

'cuda'

In [112]:
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Fine-tune (train) the pre-trained lanugage model.

In [113]:
trainer.train()

Epoch,Training Loss,Validation Loss


AttributeError: 'float' object has no attribute 'size'

In [121]:
trainer.evaluate()

Epoch,Training Loss,Validation Loss


AttributeError: 'float' object has no attribute 'size'

Use the trained model to make predictions for the test set.

In [122]:
# Write your code here. Add as many boxes as you need.
predictions = trainer.predict(tokenized_dataset["test"])
predictions

Epoch,Training Loss,Validation Loss


AttributeError: 'float' object has no attribute 'size'

In [ ]:
logits, labels = predictions.predictions, predictions.label_ids


In [ ]:
logits

In [ ]:
labels

In [ ]:
preds = np.argmax(logits, axis=-1)

In [123]:
preds

NameError: name 'preds' is not defined

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

In [124]:
# Write your code here. Add as many boxes as you need.
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [125]:
print(classification_report(labels, preds))


NameError: name 'labels' is not defined

In [ ]:
sns.heatmap(confusion_matrix(labels, preds), annot=True, fmt="5.2f", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")

# Laboratory Exercise - Bonus Task (+ 2 points)

Implement a simple machine learning pipeline to classify wine reviews as **exceptional** or not. Use TF-IDF vectorization to convert text into numerical features and train a logistic regression. Split the dataset into training and testing sets, fit the pipeline on the training data, and evaluate its performance using metrics such as precision, recall, and F1-score. Analyze the texts to find the most influential words or phrases associated with the **exceptional** wines. Use the coefficients from the logistic regression trained on TF-IDF features to identify the top positive and negative keywords for **exceptional** wines. Present these keywords in a simple table or visualization (e.g., bar chart).

In [119]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [115]:
# Write your code here. Add as many boxes as you need.
pipeline = make_pipeline(
    TfidfVectorizer(),
    RandomUnderSampler(),
    LogisticRegression()
)

In [116]:
pipeline.fit(dataset['train']['text'], dataset['train']['label'])

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('randomundersampler', RandomUnderSampler()),
                ('logisticregression', LogisticRegression())])

In [117]:
pred= pipeline.predict(dataset['test']['text'])

In [120]:
print(classification_report(dataset['test']['label'], pred))

              precision    recall  f1-score   support

           0       0.79      0.78      0.79       979
           1       0.80      0.81      0.80      1021

    accuracy                           0.80      2000
   macro avg       0.79      0.79      0.79      2000
weighted avg       0.79      0.80      0.79      2000

